In [ ]:
import pandas as pd
import calendar
import openpyxl
from openpyxl import load_workbook
import subprocess
import re
import os
import numpy as np
import collections
import csv

In [ ]:
# Prepare variables
months = list(calendar.month_name[1:])
years = ['2017', '2018', '2019', '2020']
years_test = ['2018']
states = ["A & N Islands", "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chandigarh", "Chhattisgarh", "Dadra & Nagar Haveli", "Daman & Diu", "Delhi", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jammu & Kashmir", "Jharkhand", "Karnataka", "Kerala", "Lakshadweep", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Puducherry", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"]
# states_2 = ["Chandigarh", "Chhattisgarh", "Dadra & Nagar Haveli", "Daman & Diu", "Delhi", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jammu & Kashmir", "Jharkhand", "Karnataka", "Kerala", "Lakshadweep", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Puducherry", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"]
states_test = ['Maharashtra']

In [ ]:
# Hold data frame columns
df_columns = ['year', 'month', 'state', 'district']
df_column_labels = collections.OrderedDict()
df_column_labels['year'] = 'Year'
df_column_labels['month'] = 'Month'
df_column_labels['state'] = 'State name as string'
df_column_labels['district'] = 'District name as string'


In [ ]:
def convert_to_stata_variable(input):
    STATA_LIMIT = 80        
    cur_length = len(input)    
    # Remove non alphanumeric and space/- characters
    input = re.sub(r'[^a-zA-Z0-9.\-\s]', '', input)
    input = re.sub(r'\s+-\s+', ' ', input)
    input = re.sub(r'-', '_', input) 
    # Split string into words
    words = input.split()
    # IF only word is there return it
    if len(words) == 1:
        return input
    trunc_words = []    
    for word in words:
        # Check for stop words
        if word.lower() in ['of', 'and', 'or', 'due', 'to', 'a', 'the', 'etc', 'for']:
            continue
        elif len(word) < 5:
            # Check for capitalization
            if word[0].isupper():
                trunc_words.append(word)
            else:
                trunc_words.append(word.capitalize())
        else:
            # truncate to three            
            if word[0].isupper():
                trunc_words.append(word[:4])
            else:
                trunc_words.append(word[:4].capitalize())
    return ''.join(trunc_words)

In [ ]:
# count = 0
# for key, val in df_column_labels.items():
#     count = count + 1    
#     new_label = convert_to_stata_variable(key)    
#     if len(new_label) > 80:
#         print(key)
#         print(new_label)
    

In [ ]:
def process_hmis_xls_file(state, year, month):
    row_dicts = []
    # Read xls file
    wb = load_workbook('data/converted/{}_{}_{}.xlsx'.format(year, state, month))
    ws = wb['Sheet1']    
    # Get the cell containing district names
    district_columns = {}
    district_range_left_bound = 'E10' 
    district_range_right_bound = ''   
    for cell_range in ws.merged_cells.ranges:
        excel_left_coord, excel_right_coord = cell_range.coord.split(':')
        if excel_left_coord == 'E9':
            district_range_right_bound = excel_right_coord[:-1] + '10'
            break
    district_range = openpyxl.worksheet.cell_range.CellRange('{}:{}'.format(district_range_left_bound, district_range_right_bound))
    for cell_range in ws.merged_cells.ranges:
        if cell_range.issubset(district_range):
            excel_left_coord, excel_right_coord = cell_range.coord.split(':')
            district_name = ws[excel_left_coord].value
            # Skip totals (for now)
            if district_name[0] == '_':
                continue
            else:
                district_columns[district_name] = excel_left_coord[:-2]
    for district_name, district_column in district_columns.items():
        district_data = {            
            'year': year,
            'month': month,
            'state': state,
            'district': district_name
        }
        # Hardcode row range for now        
        for row in range(12,548):
            # Construct column title
            prim_label_cell = 'C{}'.format(row)
            prim_label = ws[prim_label_cell].value
            if prim_label is None:
                label_range = openpyxl.worksheet.cell_range.CellRange('{}:{}'.format(prim_label_cell, prim_label_cell))                
                # Check in merged cells
                for cell_range in ws.merged_cells.ranges:
                    if label_range.issubset(cell_range):
                        excel_left_coord, excel_right_coord = cell_range.coord.split(':')
                        prim_label = ws[excel_left_coord].value         
            sec_label = ws['D{}'.format(row)].value        
            if sec_label != 'TOTAL':
                # Format a bit
                sec_label_formatted = re.sub(r'\d+\.\s+', '', sec_label)
                prim_label = prim_label + ' ' + sec_label_formatted
            # Convert prim_label to state specific format so that it doesn't get truncated in the future
            conv_label = convert_to_stata_variable(prim_label)
            if conv_label not in df_column_labels:
                df_column_labels[conv_label] = prim_label
            cell_value = ws['{}{}'.format(district_column, row)].value
            if cell_value == '':
                cell_value = np.nan            
            district_data[conv_label] = cell_value
        row_dicts.append(district_data)    
    return row_dicts    
        
    

In [ ]:
process_hmis_xls_file('Maharashtra', '2018', 'January')

In [ ]:
# Write a function to convert xls to xlsx
def convert_to_xlsx(state, year, month):
    # Assume that file doesn't exist
    convert_cmd = subprocess.run(["/Applications/LibreOffice.app/Contents/MacOS/soffice", "--headless", "--convert-to", "xlsx", "--outdir", "data/converted/", "data/{}_{}_{}.xls".format(year, state, month)])
    if convert_cmd.returncode != 0:
        print('Error converting file for {} {} {}'.format(state, year, month))
        return False
    return True
    

In [ ]:
convert_to_xlsx("A & N Islands", 2017, "September")

In [ ]:
for key, item in df_column_labels.items():
    if len(item) > 80:
        print(item)
        print(key)
        print(len(item))

In [ ]:
for state in states:
    state_rows = []
    for year in years:
        for month in months:
            # Check if file exists for the month
            if os.path.exists('data/{}_{}_{}.xls'.format(year, state, month)):
                # Proceed to check if xlsx file exists
                if os.path.exists('data/converted/{}_{}_{}.xlsx'.format(year, state, month)):
                    month_rows = process_hmis_xls_file(state, year, month)
                    state_rows.extend(month_rows)
                else:
                    conv_success = convert_to_xlsx(state, year, month)
                    if conv_success:
                        month_rows = process_hmis_xls_file(state, year, month)
                        state_rows.extend(month_rows)
                    else:
                        print("Unable to convert XLS file for {} {} {}".format(state, year, month))
            else:
                print("Skipping {} {} {} as no file found...".format(year, state, month))        
    # Create dataframe for values for a state across all years        
    state_df = pd.DataFrame.from_dict(state_rows)
    # Coerce all columns to numeric where possible
    cols = list(df_column_labels.keys())[4:]
    state_df[cols] = state_df[cols].apply(pd.to_numeric, axis=1)    
    state_df.to_csv('output/{}.csv'.format(state), index=False) 
    # Modify df_labels for stata
    df_stata_labels = {}
    for key, item in df_column_labels.items():
        if len(item) > 80:
            df_stata_labels[key] = item[:77] + '...'
        else:
            df_stata_labels[key] = item           
    state_df.to_stata('output/{}.dta'.format(state), write_index=False, variable_labels=df_stata_labels, version=118)
    print("{} written to file".format(state))    



In [ ]:
# Write variable labels to seperate file
label_file = open('output/labels.csv', 'w')
csvwriter = csv.writer(label_file)
csvwriter.writerow(["Stata Variable Name", "Detailed Label"])
for key, item in df_column_labels.items():
    csvwriter.writerow([key, item])
label_file.close()


In [ ]:
combined_df = pd.DataFrame()
variable_labels = {}
for state in states:
    stata_reader = pd.read_stata("output/{}.dta".format(state), iterator=True)
    state_df = pd.read_stata("output/{}.dta".format(state))
    variable_labels = {**variable_labels, **stata_reader.variable_labels()}
    combined_df = pd.concat([combined_df, state_df])    
combined_df.info()

In [ ]:
# Read label file
lab_file = open('output/labels.csv')
csvreader = csv.reader(lab_file)
stata_labels = {}
for row in csvreader:
    if len(row[1]) > 79:
        stata_labels[row[0]] = row[1][:76] + '...'
    else:
        stata_labels[row[0]] = row[1]
lab_file.close()

In [ ]:
combined_df.to_csv('output/combined.csv', index=False)

In [ ]:
combined_df.to_stata('output/combined.dta', write_index=False, variable_labels=stata_labels, version=118)